In [1]:
import coursier.MavenRepository

interp.repositories() ++= Seq(MavenRepository(
  "http://dl.bintray.com/cibotech/public"
))

import coursier.MavenRepository



In [2]:
import $ivy.`com.stripe::rainier-core:0.2.2`
import $ivy.`com.stripe::rainier-plot:0.2.2`
import $ivy.`org.scalanlp::breeze:0.13.2`

import $ivy.$                               

import $ivy.$                               

import $ivy.$                            

In [4]:
import breeze.stats.distributions._
import breeze.linalg._

import breeze.stats.distributions._

import breeze.linalg._

In [5]:
case class BanditState(
    reward: Double,
    longTermReward: List[Double],
    actions: Map[Int, Int]
)

def banditStep(
    epsilon: Double,
    reward: Int => Double)(s: BanditState): Rand[BanditState] = {
    for {
        u <- Uniform(0, 1)
        nextAction <- if (u < epsilon) {
          Multinomial(DenseVector.ones[Double](s.actions.length))
        } else {
          Rand.always(s.longTermReward.zipWithIndex.maxBy(_._1)._2)
        }
        newReward = reward(nextAction)
        prevCount = s.actions.get(nextAction).get
        newLongTermReward = s.longTermReward(nextAction) + (newReward - s.longTermReward(nextAction)) / prevCount
    } yield BanditState(newReward, 
                s.longTermReward.updated(nextAction, newLongTermReward),
                s.actions.updated(nextAction, prevCount + 1))
}

defined class BanditState
defined function banditStep

In [ ]:
def buildActions(actions: Int): Map[Int, Int] = {
    (1 to actions).map(a => a -> 0).toMap
}

def bandit(
    epsilon: Double, 
    actions: Int, 
    reward: Int => Double) = {
    
    val initState = BanditState(0.0, List.fill(10)(0.0), buildActions(actions))
    MarkovChain(initState)(banditStep(epsilon, reward))
}

In [ ]:
qs <- rnorm(10)
// The reward is selected from a N(q(A_t), 1)
def r(qa, action): Rand[Double] {
    Gaussian(mean = qa[action], sd = 1)
}

bandit(0.1, 10, r)